In [17]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
genesis_ic = wn.ic(genesis, False, 0.0)

import numpy as np
import pandas as pd
import warnings 
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [5]:
data = pd.read_csv('train.csv')
data.drop('Id', axis=1 , inplace=True)
data

,Comment,Topic
0,A few things. You might have negative- frequen...,Biology
1,Is it so hard to believe that there exist part...,Physics
2,There are bees,Biology
3,I'm a medication technician. And that's alot o...,Biology
4,Cesium is such a pretty metal.,Chemistry
...,...,...
8690,I make similar observations over the last week...,Biology
8691,You would know.,Biology
8692,Also use the correct number of sig figs,Chemistry
8693,"What about the ethical delimmas, groundbreaki...",Biology


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
CommentsToBeTokenized = data['Comment'] 
tfidf_result=tfidf.fit_transform(CommentsToBeTokenized).toarray()

In [8]:
vectorizer = TfidfVectorizer(max_features= 6000)
tfidf_result = vectorizer.fit_transform(CommentsToBeTokenized).toarray()
features = vectorizer.get_feature_names_out()
tfidf_result = pd.DataFrame(tfidf_result, columns=features)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(tfidf_result, data['Topic'], test_size= 0.2)
for k in range(1,5):
    model = KNC(n_neighbors= k )
    model.fit(x_train, y_train)
    y_predicted = model.predict(x_test)
    print("\nTesting Results:\n")
    print(classification_report(y_test, y_predicted))

C:\Users\serap\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)



Testing Results:

              precision    recall  f1-score   support

     Biology       0.81      0.36      0.50       707
   Chemistry       0.39      0.88      0.54       576
     Physics       0.62      0.17      0.27       456

    accuracy                           0.48      1739
   macro avg       0.61      0.47      0.44      1739
weighted avg       0.62      0.48      0.45      1739



C:\Users\serap\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)



Testing Results:

              precision    recall  f1-score   support

     Biology       0.44      0.96      0.60       707
   Chemistry       0.49      0.13      0.21       576
     Physics       0.95      0.08      0.15       456

    accuracy                           0.45      1739
   macro avg       0.63      0.39      0.32      1739
weighted avg       0.59      0.45      0.35      1739



C:\Users\serap\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)



Testing Results:

              precision    recall  f1-score   support

     Biology       0.71      0.38      0.49       707
   Chemistry       0.39      0.89      0.55       576
     Physics       0.75      0.09      0.15       456

    accuracy                           0.47      1739
   macro avg       0.62      0.45      0.40      1739
weighted avg       0.62      0.47      0.42      1739


Testing Results:

              precision    recall  f1-score   support

     Biology       0.60      0.26      0.36       707
   Chemistry       0.36      0.62      0.45       576
     Physics       0.32      0.31      0.32       456

    accuracy                           0.39      1739
   macro avg       0.43      0.40      0.38      1739
weighted avg       0.45      0.39      0.38      1739



C:\Users\serap\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
